In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
import pandas as pd

In [ ]:
from multiprocessing import Pool

In [ ]:
# https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/

In [ ]:
empresas_listadas = pd.read_csv('InstrumentsConsolidatedFile_20220414_1.csv', sep=';')

In [ ]:
def get_indicators(browser, path_indicators: str, path_values:str):
    df = pd.DataFrame(columns=['Ticker', 'Indicador'])

    indicadores = browser.find_element(by=By.XPATH, value=path_indicators)
    html_indicadores = indicadores.get_attribute('outerHTML')
    splited_indicadores = html_indicadores.split('data-name')
    for i in range(1, len(splited_indicadores)):
    #     print(splited_indicadores[i].split('=')[1].split('data-prefix')[0])
        df2 = {'Indicador': splited_indicadores[i].split('=')[1].split('data-prefix')[0].replace('"', '')}

        df = df.append(df2, ignore_index = True)


    valores = browser.find_element(by=By.XPATH, value=path_values)
    html_valores  = valores.get_attribute('outerHTML')
    splited_divs = html_valores.split('<div')
    # print(splited_divs)

    df_index = -1
    for i in range(1, len(splited_divs)):
    #     print(splited_divs[i])
        if ('tr w-100 d-flex justify-start asset-last' in splited_divs[i]):
    #         print('\n')
            is_header = False
            df_index +=1
            df_col = 2

        p = re.compile('>(.*)</')
        val = p.findall(splited_divs[i])

        if len(val) > 0:
            val = val[0].replace('</div>', '')
            if 'ATUAL' in val:
                is_header = True

            if is_header is True:
                df[val] = None

            else:
                df.iloc[df_index, df_col] = val
    #             print(val)
                df_col+=1
        
    return df

In [ ]:
def get_company_indicators(ticker:str):
    try:
        driver = webdriver.Firefox("/usr/local/bin/")
        driver.get(f'https://statusinvest.com.br/acoes/{ticker}')

        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,"btn-close"))).click() # fecha o popup

        button = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]') # botão de histórico

        time.sleep(0.5)
        button.click()
        time.sleep(0.5)

        df_valuation = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[2]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[2]/div/div[2]/div')
        df_endividamento = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[3]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[3]/div/div[2]/div')
        df_eficiencia = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[4]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[4]/div/div[2]/div')
        df_rentabilidade = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[5]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[5]/div/div[2]/div')
        df_crescimento = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[6]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[6]/div/div[2]/div')

        df = pd.concat([df_valuation, df_endividamento, df_eficiencia, df_rentabilidade, df_crescimento]).reset_index(drop=True)
        df['Ticker'] = ticker
        
        setor = driver.find_element(by=By.XPATH, value='/html/body/main/div[5]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong')
        setor = setor.get_attribute('outerHTML')
        df['Setor'] = setor.split('>')[1].split('<')[0]
        
        driver.close()
        
    except Exception as ex:
        print(ex)
        try:
            driver.close()
        except Exception:
            pass
        return None
    
    return df

In [ ]:
get_company_indicators('WEGE3')

In [ ]:
tickers = empresas_listadas.loc[(empresas_listadas['TckrSymb'].str.len()>= 5) \
                                & (empresas_listadas['TckrSymb'].str.len()<= 6) \
                                & (~empresas_listadas['TckrSymb'].str.contains('^[0-9]')) \
                                & (~empresas_listadas['TckrSymb'].str.endswith('34'))\
                                & (~empresas_listadas['TckrSymb'].str.endswith('39'))\
                                & ((empresas_listadas['TckrSymb'].str.endswith('3')) | (empresas_listadas['TckrSymb'].str.endswith('3')) | (empresas_listadas['TckrSymb'].str.endswith('11')))
                                & (~empresas_listadas['TckrSymb'].str.endswith('T')), 'TckrSymb']
len(tickers)

In [ ]:
dataframes_list = []
with Pool(4) as p:
    dataframes_list = (p.map(get_company_indicators, tickers.values))

In [ ]:
data = pd.concat(list(filter(lambda x: x is not None, dataframes_list)))

In [ ]:
data.to_csv('data.csv')

In [ ]:
# for ticker in tickers.values[:3]:
#     print(get_company_indicators(ticker))

In [ ]:
data